In [ ]:
from pathlib import Path
import pandas as pd
import plotly.express as px
import plotly

#need to configure to use outdir and get scenario name
data_dir = Path.cwd() / '../../time_coincident/outputs/'

#define formatting options/functions
pd.options.display.float_format = '{:,.2f}'.format
def format_currency(x): return '$ {:,.2f}'.format(x)
def format_percent(x): return '{:,.2f}%'.format(x)

In [ ]:
# Summarize generation and generation costs by generator
########################################################

generation = pd.read_csv(data_dir / 'generation.csv')
generation = generation.round(decimals=2)
generation = generation[generation['Energy_GWh_typical_yr'] > 0]

#add system power data
try:
    system_power = pd.read_csv(data_dir / 'system_power_summary.csv')
    system_power_data = 1
except:
    system_power_data = 0
if system_power_data == 1:
    system_power = system_power.rename(columns={'System_Power_GWh_per_year':'Energy_GWh_typical_yr', 'Annual_System_Power_Cost':'Total_Annual_Cost'})
    system_power = system_power.drop(columns=['load_zone'])
    system_power['generation_project'] = 'SYSTEM_POWER'

#load capacity costs
capacity_cost = pd.read_csv(data_dir / 'gen_cap.csv', usecols=['generation_project','Annual_PPA_Capacity_Cost'])

generation = generation.merge(capacity_cost, how='left', on='generation_project')

#calculate total cost by generator
generation['Total_Annual_Cost'] = generation['Annual_PPA_Energy_Cost'] + generation['Annual_Excess_Energy_Cost'] + generation['Annual_PPA_Capacity_Cost']

#append system power data to generation data
if system_power_data == 1:
    generation = generation.append(system_power, ignore_index=True).fillna(0)

#calculate total cost
total_cost = generation['Total_Annual_Cost'].sum()

#calculate total generation
generation['Total_Generation_GWh_per_year'] = generation['Energy_GWh_typical_yr'] + generation['Excess_Energy_GWh_per_yr']

#calculate percent of generation that is excess for each resource
generation['Percent_Excess'] = (generation['Excess_Energy_GWh_per_yr'] / generation['Total_Generation_GWh_per_year'] * 100).round(decimals=2)

#calculate % of supply provided by each generator
total_GWh = generation['Energy_GWh_typical_yr'].sum()
generation['Percent_of_Consumption'] = (generation['Energy_GWh_typical_yr'] / total_GWh * 100).round(decimals=2)

#calculate % of cost from each generator
generation['Percent_of_Cost'] = (generation['Total_Annual_Cost'] / total_cost * 100).round(decimals=2)

#total row
generation = generation.append(generation.sum(numeric_only=True, axis=0), ignore_index=True).fillna('Total')

#recalculate total % excess
generation.at[(generation[generation['generation_project'] == 'Total']).index[0], 'Percent_Excess'] = ((generation[generation['generation_project'] == 'Total']['Excess_Energy_GWh_per_yr']).to_numpy()[0] / (generation[generation['generation_project'] == 'Total']['Total_Generation_GWh_per_year']).to_numpy()[0]) * 100

generation = generation[['generation_project','Energy_GWh_typical_yr','Excess_Energy_GWh_per_yr','Total_Generation_GWh_per_year','Percent_Excess','Annual_PPA_Energy_Cost','Annual_PPA_Capacity_Cost','Annual_Excess_Energy_Cost','Total_Annual_Cost','Percent_of_Cost']]
generation = generation.rename(columns={'generation_project':'Generator','Energy_GWh_typical_yr':'Consumed_Generation_GWh','Excess_Energy_GWh_per_yr':'Excess_Generation_GWh','Total_Generation_GWh_per_year':'Total_Generation_GWh','Annual_Excess_Energy_Cost':'Annual_Excess_Generation_Cost'})

generation = generation.round(decimals=2)

In [ ]:
# Summarize the composition of the portfolio by generation project
##################################################################

portfolio = pd.read_csv(data_dir / 'BuildGen.csv', names=['Generator','Build_Year','MW'], skiprows=[0])
portfolio = portfolio.drop(columns=['Build_Year'])
#keep rows that are greater than 0
portfolio = portfolio[portfolio['MW'] > 0]

total_MW = portfolio['MW'].sum()

#Add percentage column
portfolio['Capacity_Percent'] = (portfolio['MW'] / total_MW * 100)
portfolio['MW'] = portfolio['MW'].round(decimals=2)

#merge generation data
portfolio = portfolio.merge(generation[['Generator','Consumed_Generation_GWh']], how='right', on='Generator').fillna(0)
#drop the total row
portfolio = portfolio.drop(portfolio.tail(1).index)

total_GWh = portfolio['Consumed_Generation_GWh'].sum()
portfolio['Energy_Percent'] = (portfolio['Consumed_Generation_GWh'] / total_GWh * 100)

#append a total column
portfolio = portfolio.append(portfolio.sum(numeric_only=True, axis=0), ignore_index=True).fillna('Total')


In [ ]:
# Summarize portfolio mix by technology
#######################################

tech_portfolio = portfolio.copy()

#split the prefix of the generator name from the rest of the string
tech_column = [i.split('_')[0] for i in tech_portfolio['Generator']]

#create a new tecnhology column based on the prefix and drop the generator name
tech_portfolio['Technology'] = tech_column
tech_portfolio = tech_portfolio.drop(columns=['Generator'])

#group by technology type and sum
tech_portfolio = tech_portfolio.groupby('Technology').sum()


In [ ]:
# Summarize RA Value by generation project
##########################################

ra_value = pd.read_csv(data_dir / 'RA_value_by_generator.csv', index_col=["Period","Generation_Project","Local_Reliability_Area","Month"])

#pivot the data
ra_value_pivot = ra_value.pivot(columns="RA_Requirement")["RA_Value"].fillna(0)
#drop any rows where all values are zero
ra_value_pivot = ra_value_pivot.loc[(ra_value_pivot != 0).any(axis=1)]
ra_value_pivot = ra_value_pivot.reset_index().rename_axis(None, axis=1)

#average the values across all months
ra_value_pivot = ra_value_pivot.groupby(["Period","Generation_Project","Local_Reliability_Area"]).mean().drop(columns=['Month']).round(decimals=1).reset_index()

# #add built capacity
ra_value_pivot = ra_value_pivot.merge(portfolio[['Generator','MW']], how='left', left_on='Generation_Project', right_on='Generator')

ra_value_pivot = ra_value_pivot[['Generation_Project','Local_Reliability_Area','MW','system_RA','local_PGE_other_RA','local_greater_bay_RA','flexible_RA']]

#append a total column
ra_value_pivot = ra_value_pivot.append(ra_value_pivot.sum(numeric_only=True, axis=0), ignore_index=True).fillna('Total')

In [ ]:
# Summarize RA Cost by requirement type
#######################################

ra_cost = pd.read_csv(data_dir / 'RA_open_position.csv')
#groupby RA requirement
ra_cost = ra_cost.groupby('RA_Requirement').mean().drop(columns=['Period','Month'])
#rename columns
ra_cost = ra_cost.rename(columns={'Open_Position_MW': 'Monthly_Open_Position_MW', 'Open_Position_Cost': 'Monthly_Open_Position_Cost'})
#calculate teh annual
ra_cost['Annual_Open_Position_Cost'] = ra_cost['Monthly_Open_Position_Cost'] * 12

In [ ]:
# Summarize Portfolio Cost
##########################

electricty_cost_df = pd.read_csv(data_dir / 'electricity_cost.csv')
energy_cost_total = electricty_cost_df['EnergyCostReal_per_MWh'].to_numpy()[0]
system_demand_mwh = electricty_cost_df['SystemDemand_MWh'].to_numpy()[0]

#calculate delivered costs
costs_itemized_df = pd.read_csv(data_dir / 'costs_itemized.csv')
generation_cost = costs_itemized_df[costs_itemized_df['Component'] == 'GenPPACostInTP']['AnnualCost_Real'].to_numpy()[0] / system_demand_mwh
storage_cost = costs_itemized_df[costs_itemized_df['Component'] == 'TotalGenCapacityCost']['AnnualCost_Real'].to_numpy()[0] / system_demand_mwh
excess_generation_cost = costs_itemized_df[costs_itemized_df['Component'] == 'ExcessGenCostInTP']['AnnualCost_Real'].to_numpy()[0] / system_demand_mwh
ra_cost = costs_itemized_df[costs_itemized_df['Component'] == 'TotalRAOpenPositionCost']['AnnualCost_Real'].to_numpy()[0] / system_demand_mwh
try:
    system_power_cost = costs_itemized_df[costs_itemized_df['Component'] == 'SystemPowerCost']['AnnualCost_Real'].to_numpy()[0] / system_demand_mwh
except:
    system_power_cost = 0

#calculate weighted component cost
#generation  = cost / Energy_GWh_typical_yr
weighted_generation = costs_itemized_df[costs_itemized_df['Component'] == 'GenPPACostInTP']['AnnualCost_Real'].to_numpy()[0] / (generation[generation['Generator'] == 'Total']['Consumed_Generation_GWh'] * 1000).to_numpy()[0]
#excess generation = cost / Excess_GWh
weighted_excess = costs_itemized_df[costs_itemized_df['Component'] == 'ExcessGenCostInTP']['AnnualCost_Real'].to_numpy()[0] / (generation[generation['Generator'] == 'Total']['Excess_Generation_GWh'] * 1000).to_numpy()[0]
#storage = cost / BuildGen ($/MW-year)
weighted_storage = costs_itemized_df[costs_itemized_df['Component'] == 'TotalGenCapacityCost']['AnnualCost_Real'].to_numpy()[0] / tech_portfolio[tech_portfolio.index == 'STORAGE']['MW'].to_numpy()[0] / 12000
weighted_RA = costs_itemized_df[costs_itemized_df['Component'] == 'TotalRAOpenPositionCost']['AnnualCost_Real'].to_numpy()[0] / tech_portfolio[tech_portfolio.index == 'Total']['MW'].to_numpy()[0] / 12000
#system power = cost / system power GWh
try:
    weighted_system = costs_itemized_df[costs_itemized_df['Component'] == 'SystemPowerCost']['AnnualCost_Real'].to_numpy()[0] / (system_power['Energy_GWh_typical_yr'].to_numpy()[0] * 1000)
except:
    weighted_system = 0

cost = {
    'Cost Component': ['Portfolio Total','Generation','Excess Generation','Storage','RA Open Position','System Power'],
    '$/MWh delivered': [energy_cost_total, generation_cost, excess_generation_cost, storage_cost, ra_cost, system_power_cost],
    'Weighted Cost': [energy_cost_total, weighted_generation, weighted_excess, weighted_storage, weighted_RA, weighted_system],
    'Unit': ['$/MWh','$/MWh','$/MWh','$/kW-mo', '$/kW-mo','$/MWh']
}

cost_summary = pd.DataFrame(cost)
cost_summary = cost_summary.round(decimals=2)

In [ ]:
# Calculate Time-coincident renewable % and annual renewable %
##############################################################

if system_power_data == 0:
    tc_percent_renewable = 100
else:
    tc_percent_renewable = 100 - (portfolio[portfolio['Generator'] == 'SYSTEM_POWER']['Energy_Percent']).to_numpy()[0]


if system_power_data == 1:
    system_power_mwh = (generation[generation['Generator'] == 'SYSTEM_POWER']['Total_Generation_GWh']).to_numpy()[0] * 1000
else:
    system_power_mwh = 0
annual_percent_renewable = (((generation[generation['Generator'] == 'Total']['Total_Generation_GWh']).to_numpy()[0] * 1000) - system_power_mwh) / system_demand_mwh
annual_percent_renewable = annual_percent_renewable * 100

percent_renewable = (f'The portfolio is {tc_percent_renewable}% renewable on a time-coincident basis and {annual_percent_renewable.round(decimals=2)}% renewable on an annual basis')

In [ ]:
# Create a heatmap of net load without storage
##############################################

total_generation = pd.read_csv(data_dir / 'dispatch.csv', usecols=['timestamp','gen_energy_source','DispatchGen_MW', 'Excess_Gen_MW'])
#drop dispatch from storage resources
total_generation = total_generation[total_generation['gen_energy_source'] != 'Electricity']
total_generation = total_generation.drop(columns=['gen_energy_source'])
#groupby timestamp and sum
total_generation = total_generation.groupby('timestamp').sum()
#convert the index to a datetimeindex
total_generation.index = pd.to_datetime(total_generation.index)
total_generation['Generation_MW'] = total_generation['DispatchGen_MW'] + total_generation['Excess_Gen_MW']
total_generation = total_generation.drop(columns=['DispatchGen_MW','Excess_Gen_MW'])

total_load = pd.read_csv(data_dir / 'load_balance.csv', usecols=['timestamp','zone_demand_mw'], index_col='timestamp', parse_dates=True)

net_load = total_load.merge(total_generation, how='left', left_index=True, right_index=True)
net_load['Gen_Bal_MW'] = net_load['Generation_MW'] - net_load['zone_demand_mw']

net_load['day'] = net_load.index.dayofyear
net_load['hour'] = net_load.index.hour

net_load_array = net_load.pivot(index='hour',columns='day',values='Gen_Bal_MW').to_numpy()

fig_net_gen = px.imshow(net_load_array, labels=dict(x="Day of Year",y='Hour of Day',color='MW'), aspect='auto', color_continuous_scale='RdBu', color_continuous_midpoint=0, title='Net Generation without storage')


In [ ]:
#Create a heatmap of excess generation after storage charging
#############################################################

excess_generation = pd.read_csv(data_dir / 'dispatch.csv', usecols=['timestamp','gen_energy_source', 'Excess_Gen_MW'])
#drop dispatch from storage resources
excess_generation = excess_generation.drop(columns=['gen_energy_source'])
#groupby timestamp and sum
excess_generation = excess_generation.groupby('timestamp').sum()
#convert the index to a datetimeindex
excess_generation.index = pd.to_datetime(excess_generation.index)

excess_generation['day'] = excess_generation.index.dayofyear
excess_generation['hour'] = excess_generation.index.hour

excess_generation_array = excess_generation.pivot(index='hour',columns='day',values='Excess_Gen_MW').to_numpy()
excess_generation_array = excess_generation_array.round(decimals=1)

fig_excess_gen = px.imshow(excess_generation_array, labels=dict(x="Day of Year",y='Hour of Day',color='MW'), aspect='auto', color_continuous_scale='Blues',range_color=[0,650], title='Excess Generation')


In [ ]:
# Calculate weighted average cost of system power in hours of negative net generation
#####################################################################################

try:
    system_power_costs = pd.read_csv(data_dir / 'system_power.csv', usecols=['timestamp','System_Power_Cost_per_MWh'], index_col='timestamp', parse_dates=True)
except FileNotFoundError:
    system_power_costs = pd.read_csv(Path.cwd() / '../../time_coincident/inputs/system_power_cost.csv', usecols=['system_power_cost'])
    system_power_costs = system_power_costs.rename(columns={'system_power_cost': 'System_Power_Cost_per_MWh'})
    system_power_costs.index = total_generation.index

undergeneration_cost = system_power_costs.merge(net_load[['Gen_Bal_MW']], how='left', left_index=True, right_index=True)
undergeneration_cost.loc[undergeneration_cost['Gen_Bal_MW'] > 0, 'Gen_Bal_MW'] = 0
undergeneration_cost['Gen_Bal_MW'] = - undergeneration_cost['Gen_Bal_MW']
undergeneration_cost['Product'] = undergeneration_cost['System_Power_Cost_per_MWh'] * undergeneration_cost['Gen_Bal_MW']
weighted_cost = undergeneration_cost['Product'].sum() / undergeneration_cost['Gen_Bal_MW'].sum()
weighted_cost_message = (f'The weighted cost of system power during periods of undergeneration is ${weighted_cost.round(decimals=2)} per MWh')

In [ ]:
# Calculate weighted average value of system power in hours of excess net generation
####################################################################################

excess_value = system_power_costs.merge(excess_generation[['Excess_Gen_MW']], how='left', left_index=True, right_index=True)

excess_value['Product'] = excess_value['System_Power_Cost_per_MWh'] * excess_value['Excess_Gen_MW']
weighted_value = excess_value['Product'].sum() / excess_value['Excess_Gen_MW'].sum()
weighted_value_message = (f'The weighted value of excess generation (at system power prices) is ${weighted_value.round(decimals=2)} per MWh')

In [ ]:
# Set up data for dispatch timeseries graph
###########################################

#load generation data
dispatch = pd.read_csv(data_dir / 'dispatch.csv', usecols=['timestamp','generation_project','DispatchGen_MW', 'Excess_Gen_MW'])

#pivot the data to wide format
excess = dispatch.pivot(index='timestamp',columns='generation_project',values='Excess_Gen_MW')
dispatch = dispatch.pivot(index='timestamp',columns='generation_project',values='DispatchGen_MW')

#round values to nearest hundredth of a MW
dispatch = dispatch.round(decimals=2)
excess = excess.round(decimals=2)

#remove any columns that are all zeros
dispatch = dispatch.loc[:, (dispatch != 0).any(axis=0)]
excess = excess.loc[:, (excess != 0).any(axis=0)]

#rename the columns by technology type
dispatch.columns = [i.split('_')[0] for i in dispatch.columns]
excess.columns = [i.split('_')[0] for i in excess.columns]

#groupby the column names
dispatch = dispatch.groupby(dispatch.columns, axis=1).sum()
excess = excess.groupby(excess.columns, axis=1).sum()
excess = excess.add_suffix('_Excess')

#need to add system power data
try:
    system_power_use = pd.read_csv(data_dir / 'system_power.csv', usecols=['timestamp','System_Power_MW'], index_col='timestamp', parse_dates=True)
    system_power_use.columns = ['SYSTEM_POWER']
    dispatch = dispatch.merge(system_power_use, how='left', left_index=True, right_index=True)

    #merge excess data
    dispatch = dispatch.merge(excess, how='left', left_index=True, right_index=True)
except:
    dispatch = dispatch.merge(excess, how='left', left_index=True, right_index=True)

#load storage charge data
charge = pd.read_csv(data_dir / 'storage_dispatch.csv', parse_dates=True)
charge = charge.pivot(index='timepoint', columns='generation_project',values='ChargeMW')
charge = charge.loc[:, (charge != 0).any(axis=0)]
charge.index = pd.to_datetime(charge.index)
charge.index = charge.index.rename('timestamp')
#rename the columns by technology type
charge.columns = [i.split('_')[0] for i in charge.columns]
charge = charge.groupby(charge.columns, axis=1).sum()
charge = charge.add_suffix('_Charge')

dispatch = dispatch.merge(charge, how='left', left_index=True, right_index=True)

#load load
load = pd.read_csv(data_dir / 'load_balance.csv', usecols=['timestamp','zone_demand_mw'], index_col='timestamp', parse_dates=True)
load.columns = ['DEMAND']

dispatch = dispatch.merge(load, how='left', left_index=True, right_index=True)


In [ ]:
# Format the currency and percent values in the tables
######################################################
portfolio['Energy_Percent'] = portfolio['Energy_Percent'].apply(format_percent)
portfolio['Capacity_Percent'] = portfolio['Capacity_Percent'].apply(format_percent)

tech_portfolio['Energy_Percent'] = tech_portfolio['Energy_Percent'].apply(format_percent)
tech_portfolio['Capacity_Percent'] = tech_portfolio['Capacity_Percent'].apply(format_percent)

generation['Percent_Excess'] = generation['Percent_Excess'].apply(format_percent)
generation['Annual_PPA_Energy_Cost'] = generation['Annual_PPA_Energy_Cost'].apply(format_currency)
generation['Annual_PPA_Capacity_Cost'] = generation['Annual_PPA_Capacity_Cost'].apply(format_currency)
generation['Annual_Excess_Generation_Cost'] = generation['Annual_Excess_Generation_Cost'].apply(format_currency)
generation['Total_Annual_Cost'] = generation['Total_Annual_Cost'].apply(format_currency)
generation['Percent_of_Cost'] = generation['Percent_of_Cost'].apply(format_percent)

cost_summary['$/MWh delivered'] = cost_summary['$/MWh delivered'].apply(format_currency)
cost_summary['Weighted Cost'] = cost_summary['Weighted Cost'].apply(format_currency)



In [ ]:
#allow the notebook to display plots in html report
plotly.offline.init_notebook_mode()

# Scenario Report

In [ ]:
scenario_name = str(Path.cwd()).split('\\')[-1]

print(f'Scenario Name: {scenario_name}')

## Portfolio Renewable Percentage

In [ ]:
print(percent_renewable)

## Portfolio Mix by Technology

In [ ]:
tech_portfolio

## Portfolio Mix by Generator

In [ ]:
portfolio

## Interactive Plot of Portfolio Dispatch and Load

In [ ]:
dispatch_columns = [i for i in dispatch.columns if (('Charge' not in i) & ('DEMAND' not in i))]
color_map = {'HYDRO':'Purple',
 'ONWIND':'Blue',
 'PV':'Yellow',
 'STORAGE':'Green',
 'HYDRO_Excess':'Plum',
 'ONWIND_Excess':'SkyBlue',
 'PV_Excess':'LemonChiffon',
 'SYSTEM_POWER':'Red'}

fig = px.area(dispatch, x=dispatch.index, y=dispatch_columns, color_discrete_map=color_map, labels={'timestamp':'Datetime','value':'MW'})
fig.layout.template = 'plotly_white'
fig.add_scatter(x=dispatch.index, y=(dispatch['DEMAND']+dispatch['STORAGE_Charge']), line=dict(color='Green', width=4), name='STORAGE_Charge')
fig.add_scatter(x=dispatch.index, y=dispatch['DEMAND'], line=dict(color='black', width=4), name='Demand')
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1d", step="day", stepmode="backward"),
            dict(count=7, label="1w", step="day", stepmode="backward"),
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(step="all")
        ])))

fig.show()

## Delivered Electricity Cost Summary

In [ ]:
cost_summary

## Generation and Cost by Generator

In [ ]:
generation

## Monthly Average RA Value by Generator

In [ ]:
ra_value_pivot

## RA Open Position Cost by Requirement

In [ ]:
ra_cost

## Hourly Net Generation Heat Map
- Shows when we are over- and under-procured
- Generation - Demand, ignoring storage charging and discharging


In [ ]:
print(weighted_cost_message)

In [ ]:
fig_net_gen.show()

## Hourly Excess Generation
- Shows how much excess generation is left over after dispatching storage

In [ ]:
print(weighted_value_message)

In [ ]:
fig_excess_gen.show()